# 4. 測定結果の振り返りと測定計画

長期測定の結果はどの程度正確に推測することができるようになったでしょうか？ここではこれまでの測定結果をまとめつつ、今後の測定計画及び結果の解釈に役立ててください。

今回も前回までの推定結果を引き継ぎましょう。
4.0.1で前回と同じ設定を引き継ぎます。続いて4.0.2を**前回のノートにコピーして**3.0.1から順に実行したのち、4.0.2を実行してください。出力結果を4.0.3.にコピーして実行してください。

In [ ]:
# @title 4.0.1. 必要設定の読み込み

#スクリプト実行に必要なパッケージ。
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit,minimize
from scipy.stats import poisson,norm
from scipy import integrate

#用いる関数を定義
def gaussian(x, a, m, s):#ガウシアン
    return a*np.exp(-0.5*(x-m)**2/s**2)
def lin(x,a,b):
    return a*x+b
def dlin(x,a,b,dx,da,db):#直線フィッティングの誤差導出用関数
    dy=(x*da)**2
    dy+=(dx*a)**2
    dy+=db**2
    dy=np.sqrt(dy)
    #print("{:10.4f}, {:10.4f}, {:10.4f}".format((x*da),dx*a,db))
    return dy
def pathToTime(path):
    time_format='"%Y-%m-%d" "%H:%M:%S"'
    temp = np.genfromtxt(path, skip_header=1025,skip_footer=1,dtype=str)
    start_text=temp[0]+' '+temp[1]
    start_time=datetime.datetime.strptime(start_text, time_format)
    end_text=temp[2]+' '+temp[3]
    end_time=datetime.datetime.strptime(end_text, time_format)
    Time=(end_time-start_time).seconds
    return Time
def significance(n, b, sigma): #有意性を返す式
    return (n-b)/np.sqrt(b+sigma**2)
def prob(Z):#有意性を「観測した事象より大きく予想から外れる確率」に解釈しなおす式
    return 1-(norm.cdf(Z)-norm.cdf(-Z))
def rebin(x,y,ndiv=3):
    bins = len(x)-1
    # ndiv
    print(len(x),x[-1],ndiv,bins%ndiv)
    if bins%ndiv != 0:
      return rebin(x[:-1],y[:-1],ndiv)
    bins_new = int(bins/ndiv)
    print(bins_new)
    dx=x[1]-x[0]
    x_new = np.linspace(x[0],x[-1], bins_new+1)
    y_new = np.array([np.sum(y[i*ndiv:(i+1)*ndiv]) for i in range(bins_new+1)])
    print(len(x_new),x_new[-1])
    return x_new, y_new
def evalResult(s,ds,n,b,db,material):
  sig_dif=(n-s-b)/np.sqrt(n+ds**2+db**2)
  significance=s/np.sqrt(b+ds**2+db**2)
  print("{:2} expected {:.2e}+/-{:.2e}, background {:.2e}+/-{:.2e}, observed {:.2e} -> diff {:.2e}σ, significance(exp) {:.2e}σ ".format(material,s,ds,b,db,n,sig_dif,significance))
def dtheta(theta,ds1=3,ds2=3,Lrt=58.3):
  flag=1
  L2=11.7
  alpha=-1*flag#cos(agnle) on target
  beta=1*flag#cos(angle) on sensor
  gamma=1*flag#cos(angle) on radiation source
  dsi=5#mm, window size of radiation source

  theta_orig=theta
  theta_orig=np.radians(theta_orig)

  vL12=np.array([0,0,Lrt+L2])
  vdL12=np.array([alpha*np.cos(np.pi/6),0,alpha*np.sin(np.pi/6)])
  vdL12*=ds1*(Lrt+L2)/Lrt
  vdLi=np.array([dsi*gamma,0,0])
  vLI=vL12+vdL12-vdLi

  vL3=np.array([-Lts*np.sin(theta_orig),0,Lts*np.cos(theta_orig)])
  vdL3=np.array([ds2*beta*np.sin(theta_orig),0,ds2*beta*np.cos(theta_orig)])
  vLO=vL3+vdL3-vdL12
  print("設定角度{:.3f} 流入ベクトル角度{:.3f} 散乱ベクトル角度{:.3f} 散乱しない粒子の流入が可能か{:b}".format(theta,np.degrees(np.arctan(vLI[0]/vLI[2])),np.degrees(np.arctan(vLO[0]/vLO[2])),np.degrees(np.arctan(vLI[0]/vLI[2]))<np.degrees(np.arctan(vLO[0]/vLO[2]))))
  if np.degrees(np.arctan(vLI[0]/vLI[2]))<np.degrees(np.arctan(vLO[0]/vLO[2])):
    return theta
  thetap=np.arccos(np.dot(vLI,vLO)/np.sqrt(np.dot(vLI,vLI)*np.dot(vLO,vLO)))
  print("最小流入可能角度{:.3f}".format(np.degrees(thetap)))
  return abs(np.degrees(thetap)-theta)

#実験データなど、Googleドライブにあるファイルにアクセスしたい場合に実行する。自分のアカウントで要認証。
from google.colab import drive
drive.mount('/content/drive')

#ファイルを格納するディレクトリを指定、"b3exp_alpha"という名前のディレクトリをマイドライブ直下に作ることを推奨
basePath='/content/drive/MyDrive/b3exp_alpha/'

In [ ]:
# @title 4.0.2. 各種設定の引継ぎ（day2にコピーして使う！）
print(f"a_ev={a_ev:5.2e}")
print(f"da_ev={da_ev:5.2e}")
print(f"b_ev={b_ev:5.2e}")
print(f"db_ev={db_ev:5.2e}")
print(f"drt={drt:5.2e}")
print(f"drs={drs:5.2e}")
print(f"Lrs={Lrs:5.2e}")
print(f"Lrt={Lrt:5.2e}")
print(f"Lts={Lts:5.2e}")
print(f"B0={B0:5.2e}")
print(f"dB0={dB0:5.2e}")
print(f"RAl={RAl:5.2e}")
print(f"dRAl={dRAl:5.2e}")
print(f"RAu={RAu:5.2e}")
print(f"dRAu={dRAu:5.2e}")
print(f"NAl={NAl:5.2e}")
print(f"dNAl={dNAl:5.2e}")
print(f"NAu={NAu:5.2e}")
print(f"dNAu={dNAu:5.2e}")
print(f"AAl={AAl:5.2e}")
print(f"AAu={AAu:5.2e}")
print(f'Al_file_list={Al_file_list}')
print(f'Au_file_list={Au_file_list}')
print(f"basePath=\"{basePath}\"")

In [3]:
# @title 4.0.3. 必要設定のコピー場所(day2からコピーしてくる)

#以下はダミーデータ。上書きすること。
a_ev=6.19e-03
da_ev=5.58e-05
b_ev=1.29e-01
db_ev=3.18e-02
drt=3.00e+00
drs=3.00e+00
Lrs=1.30e+02
Lrt=6.00e+01
Lts=7.00e+01
B0=2.69e+06
dB0=1.79e+04
RAl=3.92e+00
dRAl=4.82e-01
RAu=2.31e+00
dRAu=1.51e-01
NAl=8.82e+19
dNAl=2.90e+18
NAu=3.36e+19
dNAu=8.88e+17
AAl=4.33e+00
AAu=1.60e+02
Al_file_list=[['300sec20degAl.csv', 20], ['60sec10degAl.csv', 10]]
Au_file_list=[['Au50deg68h.csv', 50]]

basePath="/content/drive/MyDrive/b3exp_alpha/"

In [ ]:
# @title　4.0.4. 結果の解釈と計画
theta=1.00e+01 #必要に応じて編集する
t=7.20e+03 #必要に応じて編集する
bAl=[10,10]#Alの測定における背景事象推定数を測定した回数入れる
dbAl=[10,10]#Alの測定における背景事象推定数誤差を測定した回数入れる
bAu=[10]#Auの測定における背景事象推定数を測定した回数入れる
dbAu=[10]#Auの測定における背景事象推定数誤差を測定した回数入れる
rangeAu=[400,600]#Auの信号領域
rangeAl=[0,400]#Alの信号領域
Nrebin=20#rebinする際の分割数

S1=np.pi*drt**2
S2=np.pi*drs**2
dOmega_RtoS=S2/L**2
dOmega_RtoT=S1/Lrt**2
dOmega_TtoS=S2/Lts**2
B=B0*dOmega_RtoT/4/np.pi
dB=dB0*dOmega_RtoT/4/np.pi
dsigmadOmegaAl=AAl*(np.sin(np.radians(theta)/2))**-4
dsigmadOmegaAu=AAu*(np.sin(np.radians(theta)/2))**-4
dtheta_exp=dtheta(theta)

theta_Al=[]
dtheta_Al=[]
dsdW_Al=[]
Nmes_Al=[]
Nexp_Al=[]
dNexp_Al=[]
fig_list=[]


x=np.arange(0,1024)#x軸

for file_i,b,db in zip(Al_file_list,bAl,dbAl) :
  Al_file=np.genfromtxt(basePath+file_i[0], skip_footer=6)
  theta_Al.append(file_i[1])
  dtheta_Al.append(dtheta(file_i[1]))
  Nsig=np.sum(Al_file[rangeAl[0]:rangeAl[1]])
  tmes=pathToTime(basePath+file_i[0])
  dsdW_Al.append(Nsig/(B*tmes*NAl*1e-26*dOmega_TtoS))
  Nmes_Al.append(Nsig)
  Nexp_Al.append(B*tmes*NAl*1e-26*AAl*(np.sin(np.radians(file_i[1])/2))**-4*dOmega_TtoS)
  dNexp_Al.append(B*tmes*NAl*1e-26*AAl*(np.sin(np.radians(file_i[1])/2))**-4*dOmega_TtoS*np.sqrt((dB/B)**2+(1/np.sqrt(NAl)**2)+(dNAl/NAl)**2))
  print("測定数　{:.2f} 測定時間{:.2f}".format(Nsig,tmes))
  evalResult(Nexp_Al[-1],dNexp_Al[-1],Nmes_Al[-1],b,db,'Al')

  fig_i = plt.figure(figsize=(6, 6))
  fig_list=np.append(fig_list,fig_i)
  ax = fig_list[-1].add_subplot(111)
  xrebin,Al_file=rebin(x,Al_file,Nrebin)
  dn=np.where(Al_file > 0, np.sqrt(Al_file), 1.0)
  parSIGpeak,covSIGpeak= curve_fit(gaussian, xrebin[(xrebin>rangeAl[0]) & (rangeAl[1]>xrebin)], Al_file[(xrebin>rangeAl[0]) & (rangeAl[1]>xrebin)], p0=(np.max(Al_file), np.argmax(Al_file), 100), sigma=dn[(xrebin>rangeAl[0]) & (rangeAl[1]>xrebin)])
  perrSIGpeak = np.sqrt(np.diag(covSIGpeak))
  mat = np.vstack((parSIGpeak,perrSIGpeak)).T
  df = pd.DataFrame(mat,index=("Constant", "Mean", "Sigma"), columns=("Estimate", "Std. error"))
  print(df)
  y = gaussian(xrebin, *parSIGpeak)
  _ = ax.hist(xrebin, bins=len(xrebin)-1, weights=Al_file, range=(0, 1024), alpha=0.5, label='Al data {:.0f}$^\circ$ taking in {:.0f} sec\ntotal count {:.0f}'.format(file_i[1],tmes,Nsig))
  _ = ax.plot(xrebin, y, 'g-', label='fit')
  plt.xlabel('MCA channel [ch]', fontsize=18, fontfamily='serif')#x軸の名前
  plt.ylabel('Event count', fontsize=18, fontfamily='serif')#y軸の名前
  plt.legend(loc='upper right')


theta_Au=[]
dtheta_Au=[]
Nmes_Au=[]
Nexp_Au=[]
dNexp_Au=[]
dsdW_Au=[]
for file_i,b,db in zip(Au_file_list,bAu,dbAu) :
  Au_file=np.genfromtxt(basePath+file_i[0], skip_footer=6)
  theta_Au.append(file_i[1])
  dtheta_Au.append(dtheta(file_i[1]))
  Nsig=np.sum(Au_file[rangeAu[0]:rangeAu[1]])
  tmes=pathToTime(basePath+file_i[0])
  dsdW_Au.append(Nsig/(B*tmes*NAu*1e-26*dOmega_TtoS))
  Nmes_Au.append(Nsig)
  Nexp_Au.append(B*tmes*NAu*1e-26*AAu*(np.sin(np.radians(file_i[1])/2))**-4*dOmega_TtoS)
  dNexp_Au.append(B*tmes*NAu*1e-26*AAu*(np.sin(np.radians(file_i[1])/2))**-4*dOmega_TtoS*np.sqrt((dB/B)**2+(1/np.sqrt(NAu)**2)+(dNAu/NAu)**2))
  print("測定数　{:.2f} 測定時間{:.2f}".format(Nsig,tmes))
  evalResult(Nexp_Au[-1],dNexp_Au[-1],Nmes_Au[-1],b,db,'Au')

  fig_i = plt.figure(figsize=(6, 6))
  fig_list=np.append(fig_list,fig_i)
  ax = fig_list[-1].add_subplot(111)
  xrebin,Au_file=rebin(x,Au_file,Nrebin)
  dn=np.where(Au_file > 0, np.sqrt(Au_file), 1.0)
  parSIGpeak,covSIGpeak= curve_fit(gaussian, xrebin[(xrebin>rangeAu[0]) & (rangeAu[1]>xrebin)], Au_file[(xrebin>rangeAu[0]) & (rangeAu[1]>xrebin)], p0=(np.max(Au_file), np.argmax(Au_file), 100), sigma=dn[(xrebin>rangeAu[0]) & (rangeAu[1]>xrebin)])
  perrSIGpeak = np.sqrt(np.diag(covSIGpeak))
  mat = np.vstack((parSIGpeak,perrSIGpeak)).T
  df = pd.DataFrame(mat,index=("Constant", "Mean", "Sigma"), columns=("Estimate", "Std. error"))
  print(df)
  y = gaussian(xrebin, *parSIGpeak)
  _ = ax.hist(xrebin, bins=len(xrebin)-1, weights=Au_file, range=(0, 1024), alpha=0.5, label='Au data {:.0f}$^\circ$ taking in {:.0f} sec\ntotal count {:.0f}'.format(file_i[1],tmes,Nsig))
  _ = ax.plot(xrebin, y, 'g-', label='fit')
  plt.xlabel('MCA channel [ch]', fontsize=18, fontfamily='serif')#x軸の名前
  plt.ylabel('Event count', fontsize=18, fontfamily='serif')#y軸の名前
  plt.legend(loc='upper right')

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)
x=np.arange(5,90)#プロットするx軸のbinを設定
fAl=AAl*(np.sin(np.radians(x)/2))**-4
fAu=AAu*(np.sin(np.radians(x)/2))**-4
ax.plot(x, fAl, 'b-',label='Al theory')
ax.plot(x, fAu, 'r-',label='Au theory')
plt.errorbar(theta,dsigmadOmegaAl,xerr=dtheta_exp,marker="*",color="b",label='Al expected')#理論値Al
plt.errorbar(theta,dsigmadOmegaAu,xerr=dtheta_exp,marker="*",color="r",label='Au expected')#理論値Al
plt.errorbar(theta_Al,dsdW_Al,xerr=dtheta_Al,fmt="o",color="b",label='Al data')#測定値Al
plt.errorbar(theta_Au,dsdW_Au,xerr=dtheta_Au,fmt="o",color="r",label='Au data')#測定値Au
plt.yscale('log')  # ログスケール
ax.set_xlabel(r'$\theta$ (deg.)')
ax.set_ylabel(r'$d\sigma/d\Omega$ (fm$^2$)')
plt.legend(loc='upper right')

N_expectedAl=B*t*NAl*1e-26*dsigmadOmegaAl*dOmega_TtoS
dN_expectedAl=B*t*NAl*1e-26*dsigmadOmegaAl*dOmega_TtoS*np.sqrt((dB/B)**2+(1/np.sqrt(NAl)**2)+(dNAl/NAl)**2)
print("推定測定回数(Al)={:10.1e}+/-{:10.1e}".format(N_expectedAl,dN_expectedAl))
N_expectedAu=B*t*NAu*1e-26*dsigmadOmegaAu*dOmega_TtoS
dN_expectedAu=B*t*NAu*1e-26*dsigmadOmegaAu*dOmega_TtoS*np.sqrt((dB/B)**2+(1/np.sqrt(NAu)**2)+(dNAu/NAu)**2)
print("推定測定回数(Au)={:10.1e}+/-{:10.1e}".format(N_expectedAu,dN_expectedAu))

以下のセルと使いやすい方を使ってください。

In [ ]:
# @title 4.1.1. 計画用
theta_plan=60
t_plan=4*24*60*60
bAl=True
drt_updated=3
drs_updated=3
Lrt_updated=58.3

Ss=np.pi*drs_updated**2 #センサー手前のウィンドウ面積
dOmega_TtoS=Ss/Lts**2 #ターゲットからセンサーまでの見込み角

theta_x=np.arange(1,90)#プロットするx軸のbinを設定

B_update=B*(drt_updated/drt)**2*(Lrt_updated/Lrt)**-2

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(111)
fAl=AAl*(np.sin(np.radians(theta_x)/2))**-4
fAu=AAu*(np.sin(np.radians(theta_x)/2))**-4
ax.plot(theta_x, fAl, 'b-',label='Al theory')
ax.plot(theta_x, fAu, 'r-',label='Au theory')
plt.yscale('log')  # ログスケール
ax.set_xlabel(r'$\theta$ (deg.)')
ax.set_ylabel(r'$d\sigma/d\Omega$ (fm$^2$)')

b=1
db=1

dtheta_plan=dtheta(theta_plan,drt_updated,drs_updated,Lrt_updated)
if(bAl):
  dsigmadOmegaAl=AAl*(np.sin(np.radians(theta_plan)/2))**-4
  N_expected=B_update*t_plan*NAl*1e-26*dsigmadOmegaAl*dOmega_TtoS+b
  if dtheta==theta_plan:
    dN_expected=0
  else:
    ddsigmadOmegaAl=AAl*(np.sin(np.radians(theta_plan-dtheta_plan)/2))**-4
    dN_expected=B_update*t_plan*NAl*1e-26*ddsigmadOmegaAl*dOmega_TtoS
  dsdW_plan=N_expected/(B_update*t_plan*NAl*1e-26*dOmega_TtoS)
  ddsdW_plan=dsdW_plan-dN_expected/(B_update*t_plan*NAl*1e-26*dOmega_TtoS)
else:
  dsigmadOmegaAu=AAu*(np.sin(np.radians(theta_plan)/2))**-4
  N_expected=B_update*t_plan*NAu*1e-26*dsigmadOmegaAu*dOmega_TtoS+b
  if dtheta==theta_plan:
    dN_expected=0
  else:
    ddsigmadOmegaAu=AAu*(np.sin(np.radians(theta_plan-dtheta_plan)/2))**-4
    dN_expected=B_update*t_plan*NAu*1e-26*ddsigmadOmegaAu*dOmega_TtoS
  dsdW_plan=N_expected/(B_update*t_plan*NAu*1e-26*dOmega_TtoS)
  ddsdW_plan=dsdW_plan-dN_expected/(B_update*t_plan*NAu*1e-26*dOmega_TtoS)


dN_expected=np.sqrt(dN_expected**2+db**2)

plt.errorbar(theta_Al,dsdW_Al,xerr=dtheta_Al,fmt="o",color="b",label='Al data')#測定値Al
plt.errorbar(theta_Au,dsdW_Au,xerr=dtheta_Au,fmt="o",color="r",label='Au data')#測定値Au
plt.errorbar(theta_plan,dsdW_plan,xerr=dtheta_plan,yerr=abs(ddsdW_plan),fmt="*",label='expected')#推定値
plt.legend(loc='upper right')